# Simulating Single Phase Transport 

The point of an OpenPNM simulation is ultimately to compute some transport process.  In this notebook we will cover the following subjects:

- Defining conductance 
- Settings boundary conditions 


Start by defining a network.  We'll use the ``Demo`` class which happens to include all the geometrical pore-scale models already. 

In [2]:
import numpy as np
import openpnm as op
pn = op.network.Demo(shape=[5, 5, 1], spacing=5e-5)
print(pn)


══════════════════════════════════════════════════════════════════════════════
net : <openpnm.network.Demo at 0x25645ed9b30>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.coordination_number                                          25 / 25
  2  pore.coords                                                       25 / 25
  3  pore.diameter                                                     25 / 25
  4  pore.max_size                                                     25 / 25
  5  pore.seed                                                         25 / 25
  6  pore.volume                                                       25 / 25
  7  throat.conns                                                      40 / 40
  8  throat.cross_sectional_area                                       40 / 40
  9  

## Define Phase Viscosity

To fully illustrate the process of performing transport calculations, we'll use an empty ``Phase`` object and add all the needed properties manually:

In [3]:
water = op.phase.Phase(network=pn)

Let's assume that we are interested in pressure driven flow. This requires knowing the viscosity of the phase, so let's add a pore-scale model for computing the viscosity of water:

In [4]:
water.add_model(propname='pore.viscosity',
                model=op.models.phase.viscosity.water_correlation)
print(water)


══════════════════════════════════════════════════════════════════════════════
phase_01 : <openpnm.phase.Phase at 0x2564c657180>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.pressure                                                     25 / 25
  2  pore.temperature                                                  25 / 25
  3  pore.viscosity                                                    25 / 25
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.all                                                               25
  2  throat.all                                                             40
―

And we can check the individual values to verify they make sense:

In [5]:
print(water['pore.viscosity'])

[0.00089319 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319
 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319
 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319
 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319
 0.00089319]


## Basic Conductance Calculation

Determining the conductance of the conduits between each pair of connected pores is the most important part of performing a simulation. The details of conductance models are covered elsewhere. [](XXX)  For this demonstration will assume the very simplest case where all pressure loss occurs in the throats.

Recall the Hagan-Poiseuille equation for fluid flow through a cylindrical tube:

$$ Q = \frac{\pi R^4}{8 \mu L} \Delta P$$

where $R$ and $L$ are the radius and length of the throat, and $\mu$ is the viscosity of the fluid.  Together this prefactor can be referred to as the hydraulic conductance, $g_h$, giving:

$$ Q = g_h \Delta P $$

So the aim is the compute values of $g_h$ for each throat.  We start by doing this manually:

In [6]:
R = pn['throat.diameter']/2
L = pn['throat.length']
mu = water['throat.viscosity']  # See ProTip below
water['throat.hydraulic_conductance'] = np.pi*R**4/(8*mu*L)
print(water['throat.hydraulic_conductance'])

[1.00220815e-15 1.05133682e-15 2.65744048e-15 2.69017014e-15
 9.59190878e-15 1.36708286e-14 6.66830487e-14 7.08737841e-14
 1.99903177e-15 2.00595117e-15 4.79376978e-15 4.31484439e-15
 3.19680625e-15 2.81277856e-15 5.16906031e-15 4.79857420e-15
 8.55176371e-15 1.52083335e-15 1.23450963e-15 2.47324217e-15
 9.86091431e-15 9.79143235e-16 3.16100872e-14 2.89691924e-15
 3.71177597e-14 1.01109214e-14 1.90884769e-15 2.45494330e-14
 5.38067275e-15 7.13185895e-15 2.56325956e-14 1.72969382e-15
 2.28431461e-14 4.24865495e-15 5.99267859e-15 9.64675766e-15
 2.86021000e-15 1.49648281e-15 2.35465588e-15 9.68639559e-15]


```{tip} Phase can do auotmatic interpolation to get throat values
  The ``Phase`` class has a special ability to interpolate pore values to throats, and vice-versa. In PNM simulations all the balances are solved for each pore, so the thermodynamic properties like temperature, pressure, etc. are all defined on pores. Consequently, the physical properties are also defined in pores, like viscosity. However, as shown above we often want viscosity values in the throats. OpenPNM provides a shortcut for this, such that if you request a throat property that does not exist, it will attempt to fetch the pores values and do a linear interpolation of values to produce an array of throat values.  There is also a function for this, ``water.interpolate_data('throat.viscosity')``, but the ``water['throat.viscosity']`` shortcut is very convenient. The automatic interpolation can be disabled in the `phase.settings`.
```

In [7]:
water.interpolate_data('throat.viscosity')

array([0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319])

In [8]:
water['throat.viscosity']

array([0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319])

## Create Algorithm Object

OpenPNM contains a variety of ``Algorithm`` classes in the ``openpnm.algorithms`` module.  Let's initialize a ``StokesFlow`` algorithm, since this simulates pressure driven flow through the network. 

In [9]:
sf = op.algorithms.StokesFlow(network=pn, phase=water)
print(sf)


══════════════════════════════════════════════════════════════════════════════
stokes_01 : <openpnm.algorithms.StokesFlow at 0x2564c673f40>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.bc.rate                                                       0 / 25
  2  pore.bc.value                                                      0 / 25
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.all                                                               25
  2  throat.all                                                             40
―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

## Assign Boundary Conditions

As can be seen in the print-out above there are predefined ``'pore.bc'`` arrays, but they contain no valid values, meaning they are all ``nans``. Once we set some boundary conditions, this will change.  Let's apply pressure BCs on one side of the network, and rate BCs on the other:

In [10]:
sf.set_value_BC(pores=pn.pores('left'), values=100_000)
sf.set_rate_BC(pores=pn.pores('right'), rates=1e-10)
print(sf)


══════════════════════════════════════════════════════════════════════════════
stokes_01 : <openpnm.algorithms.StokesFlow at 0x2564c673f40>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.bc.rate                                                       5 / 25
  2  pore.bc.value                                                      5 / 25
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.all                                                               25
  2  throat.all                                                             40
―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

```{tip} All boundary conditions are preceeded with 'pore.bc'
  All boundary conditions are stored as 'pore.bc.<type>', which means that OpenPNM's dictionary lookup tricks can be used to see all types and values of bc's using: `sf['pore.bc']` which will return a `dict`. This can be used as shown below:
```

In [11]:
print(sf['pore.bc'].keys())

dict_keys(['rate', 'value'])


Now we can see there are 5 valid values of each type.  The ``sf`` algorithm will look for ``'throat.hydraulic_conductance'`` on ``water`` by default, so we can just ``run``:

In [12]:
soln = sf.run()

The ``run`` method solves the mass balance around each pore and computes the pressure within each pore that is required to sustain the flow defined by the boundary conditions.  The ``soln`` object that is returned is a dictionary with the key corresponding to the quantity that was solved for.  

In [13]:
print(soln)

{'pore.pressure': SteadyStateSolution([100000.        , 100000.        , 100000.        ,
                     100000.        , 100000.        , 111994.91786881,
                     109021.80858086, 106007.58742924, 105515.18852917,
                     104499.41168208, 126513.73814012, 120297.06671979,
                     113401.97906307, 115761.92671225, 117822.18393838,
                     132725.59341895, 133551.80327216, 120247.96545073,
                     129309.20272742, 135160.97831439, 148957.44526658,
                     155574.17873369, 168444.33285769, 161719.71531674,
                     148786.89313432])}


The reason for the dict format is to provide a consistent API for single components and multiphysics, where multiple different quanties might be solved for.  However, these ``'pore.pressure'`` values are also written to the dictionary of the algorithm object as well:

In [14]:
print(sf)


══════════════════════════════════════════════════════════════════════════════
stokes_01 : <openpnm.algorithms.StokesFlow at 0x2564c673f40>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.bc.rate                                                       5 / 25
  2  pore.bc.value                                                      5 / 25
  3  pore.initial_guess                                                25 / 25
  4  pore.pressure                                                     25 / 25
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.all                                                        

Finally we can look at how much pressure was required in the "right" pores to meet the required flow rate:

In [15]:
sf['pore.pressure'][pn.pores('right')]

array([148957.44526658, 155574.17873369, 168444.33285769, 161719.71531674,
       148786.89313432])

So we can see that 150 kPa was required to accomplish the requested flow. 

## Rigorous Conductance Calculation

The above demonstration used a very simplistic conductance calculation.  It was also stated that computing conductance is the most important part of doing a PNM simulation.  To finish this notebook, we'll look more closely at this process.

### Manual Method
Let's print the network object again:

In [16]:
print(pn)


══════════════════════════════════════════════════════════════════════════════
net : <openpnm.network.Demo at 0x25645ed9b30>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.coordination_number                                          25 / 25
  2  pore.coords                                                       25 / 25
  3  pore.diameter                                                     25 / 25
  4  pore.max_size                                                     25 / 25
  5  pore.seed                                                         25 / 25
  6  pore.volume                                                       25 / 25
  7  throat.conns                                                      40 / 40
  8  throat.cross_sectional_area                                       40 / 40
  9  

Note the ``'throat.hydraulic_size_factors'`` array.  This is computed by a pore-scale model on the ``Demo`` network.  This computation is more rigorous in the following ways:

1. The conductance of each half pore and that throat is considered.
1. The throat length is computed carefully by accounting for the 'lens' between the intersection of the spherical pore bodies and the cylindrical throat.
1. The net cross-sectional area of the pores are computed by integrating between the pore center and the pore-throat intersection point

The conductance of each element in the conduit is returned as an *Nt-by-3* array, where columns 1 and 3 contain the hydraulic conductance of the half pore on either end of the throat, and the column 1 contains the throat conductance.

In [17]:
pn['throat.hydraulic_size_factors']

array([[7.17224268e-17, 8.95163251e-19, 3.41740957e-17],
       [3.41740957e-17, 9.39044534e-19, 8.24868939e-17],
       [1.25531646e-16, 2.37360178e-18, 6.67669730e-17],
       [6.67669730e-17, 2.40283561e-18, 1.29499002e-16],
       [1.77366727e-16, 8.56740609e-18, 1.85484559e-16],
       [2.00896764e-16, 1.22106603e-17, 3.02192896e-16],
       [6.01031396e-16, 5.95606954e-17, 6.03870375e-16],
       [6.09032656e-16, 6.33038224e-17, 6.44196320e-16],
       [1.03665992e-16, 1.78551708e-18, 5.54807336e-17],
       [5.54807336e-17, 1.79169742e-18, 1.04696234e-16],
       [1.54582938e-16, 4.28175177e-18, 1.03819232e-16],
       [1.03819232e-16, 3.85397995e-18, 1.10662774e-16],
       [1.74675531e-16, 2.85535840e-18, 6.83639178e-17],
       [6.83639178e-17, 2.51234835e-18, 1.35228170e-16],
       [1.77395616e-16, 4.61695788e-18, 1.05679867e-16],
       [1.05679867e-16, 4.28604303e-18, 1.46233002e-16],
       [1.49269368e-16, 7.63835793e-18, 2.28705429e-16],
       [1.05688678e-16, 1.35839

This data is called the ``size factor`` because it is purely the geometrical information required for the computation of the hydraulic conductance.  So the Hagan-Poisseiulle equation for each element is written as:

$$ Q = \frac{F_h}{\mu} \Delta P = g_h \Delta P$$

Note that both the $\frac{\pi R^4}{8}$ term and $L$ have been rolled into the $F_h$ value.

The total conductance of the pore-throat-pore conduit can be found as the sum of three resistors in series.  Since we have conductance values, we add the inverses, and invert again.  The full expression for the hydraulic conductance between pores i and j, through throat k, is:


$$ Q = \bigg( \frac{\mu}{F_{h, i}} + \frac{\mu}{F_{h, k}} + \frac{\mu}{F_{h, j}} \bigg) ^ {-1} \Delta P $$

This can be computed by hand:

In [18]:
F_h = water['throat.hydraulic_size_factors']
water['throat.hydraulic_conductance'] = (mu * (1/F_h).sum(axis=1))**(-1)

In [19]:
water['throat.hydraulic_conductance']

array([9.64890861e-16, 1.01200775e-15, 2.52019344e-15, 2.55102875e-15,
       8.76379447e-15, 1.24146226e-14, 5.56745682e-14, 5.89529400e-14,
       1.90491669e-15, 1.91150861e-15, 4.48459671e-15, 4.02523528e-15,
       3.02123137e-15, 2.66531163e-15, 4.83218689e-15, 4.48520822e-15,
       7.88493570e-15, 1.45652099e-15, 1.17908308e-15, 2.34297724e-15,
       9.01642545e-15, 9.42596730e-16, 2.76479432e-14, 2.74408781e-15,
       3.22015299e-14, 9.24896606e-15, 1.81841007e-15, 2.17482514e-14,
       5.02794728e-15, 6.61517491e-15, 2.26770426e-14, 1.64158955e-15,
       2.02647323e-14, 3.96006729e-15, 5.56764422e-15, 8.86707390e-15,
       2.70972049e-15, 1.43348763e-15, 2.22665273e-15, 8.86413983e-15])

In [20]:
sf = op.algorithms.StokesFlow(network=pn, phase=water)
sf.set_value_BC(pores=pn.pores('left'), values=100_000)
sf.set_rate_BC(pores=pn.pores('right'), rates=1e-10)
soln = sf.run()
sf['pore.pressure'][pn.pores('right')]

array([153381.08855867, 160388.98970566, 173533.16165927, 166629.27675144,
       153163.58781883])

As can be seen the numbers are about the same as with the simple case, but should be somewhat more correct.  In fact, these above pressures are a bit higher, which is because the total conductance of the conduit is lower due to the inclusion of the pore body lengths into the total length, compared to above where only the throat length was included.

### Pore-Scale Model Method

Instead of computing the hydraulic conductance manually as done above, there is a pore-scale model available:

In [21]:
water.add_model(propname='throat.hydraulic_conductance',
                model=op.models.physics.hydraulic_conductance.generic_hydraulic)

In [22]:
sf = op.algorithms.StokesFlow(network=pn, phase=water)
sf.set_value_BC(pores=pn.pores('left'), values=100_000)
sf.set_rate_BC(pores=pn.pores('right'), rates=1e-10)
soln = sf.run()
sf['pore.pressure'][pn.pores('right')]

array([153381.08855867, 160388.98970566, 173533.16165927, 166629.27675144,
       153163.58781883])

Which gives exactly the same result, without have to manually deal with the conductances-in-series calculation.